In [ ]:
!touch submission.csv

In [ ]:
import sys
sys.path.append("/kaggle/input/pip-install-nlp-mit")

In [ ]:
!pip install "/kaggle/input/worddifficulty/py_readability_metrics-1.4.5-py3-none-any.whl"

In [ ]:
# !pip install "/kaggle/input/autocorrect/autocorrect-2.6.1.tar"
!pip install "/kaggle/input/pyspellchecker/pyspellchecker-0.7.2-py3-none-any.whl"

In [ ]:
from typing import List
import numpy as np
import pandas as pd
import warnings
import logging
import os
import shutil
import json
import transformers
from transformers import AutoModel, AutoTokenizer, AutoConfig, AutoModelForSequenceClassification
from transformers import DataCollatorWithPadding
from datasets import Dataset,load_dataset, load_from_disk
from transformers import TrainingArguments, Trainer
from datasets import load_metric, disable_progress_bar
from sklearn.metrics import mean_squared_error
import torch
from sklearn.model_selection import KFold, GroupKFold
from tqdm import tqdm

import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.tokenize.treebank import TreebankWordDetokenizer
from collections import Counter
import spacy
import re
from autocorrect import Speller
from spellchecker import SpellChecker
import lightgbm as lgb

warnings.simplefilter("ignore")
logging.disable(logging.ERROR)
os.environ["TOKENIZERS_PARALLELISM"] = "false"
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 
disable_progress_bar()
tqdm.pandas()

In [ ]:
def seed_everything(seed: int):
    import random, os
    import numpy as np
    import torch
    
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True
    
seed_everything(seed=42)

In [ ]:
class CFG:
    model_name = "another-bert"
    learning_rate=1.5e-5
    weight_decay=0.02
    hidden_dropout_prob=0.007
    attention_probs_dropout_prob=0.007
    num_train_epochs=5
    n_splits=4
    batch_size=12
    
    random_seed=42
    save_steps=20
    max_length=512

In [ ]:
MODEL_DIR = '/kaggle/input/commitlit-deberta-v3-large-masked'
INIT_MODEL = f"{MODEL_DIR}/content/{CFG.model_name}/fold_0"

## Dataload

In [ ]:
DATA_DIR = "/kaggle/input/commonlit-evaluate-student-summaries/"

prompts_train = pd.read_csv(DATA_DIR + "prompts_train.csv")
prompts_test = pd.read_csv(DATA_DIR + "prompts_test.csv")
summaries_train = pd.read_csv(DATA_DIR + "summaries_train.csv")
summaries_test = pd.read_csv(DATA_DIR + "summaries_test.csv")
sample_submission = pd.read_csv(DATA_DIR + "sample_submission.csv")

## Preprocess

[Using features]

- Text Length
- Length Ratio
- Word Overlap
- N-grams Co-occurrence
  - count
  - ratio
- Quotes Overlap
- Grammar Check
  - spelling: pyspellchecker

In [ ]:
class Preprocessor:
    def __init__(self, 
                model_name: str,
                ) -> None:
        self.tokenizer = AutoTokenizer.from_pretrained(INIT_MODEL)
        self.twd = TreebankWordDetokenizer()
        self.STOP_WORDS = set(stopwords.words('english'))
        
        self.spacy_ner_model = spacy.load('en_core_web_sm',)
        self.speller = Speller(lang='en')
        self.spellchecker = SpellChecker() 
        
    def word_overlap_count(self, row):
        """ intersection(prompt_text, text) """        
        def check_is_stop_word(word):
            return word in self.STOP_WORDS
        
        prompt_words = row['prompt_tokens']
        summary_words = row['summary_tokens']
        if self.STOP_WORDS:
            prompt_words = list(filter(check_is_stop_word, prompt_words))
            summary_words = list(filter(check_is_stop_word, summary_words))
        return len(set(prompt_words).intersection(set(summary_words)))
            
    def ngrams(self, token, n):
        # Use the zip function to help us generate n-grams
        # Concatentate the tokens into ngrams and return
        ngrams = zip(*[token[i:] for i in range(n)])
        return [" ".join(ngram) for ngram in ngrams]

    def ngram_co_occurrence(self, row, n: int) -> int:
        # Tokenize the original text and summary into words
        original_tokens = row['prompt_tokens']
        summary_tokens = row['summary_tokens']

        # Generate n-grams for the original text and summary
        original_ngrams = set(self.ngrams(original_tokens, n))
        summary_ngrams = set(self.ngrams(summary_tokens, n))

        # Calculate the number of common n-grams
        common_ngrams = original_ngrams.intersection(summary_ngrams)
        return len(common_ngrams)
    
    def ner_overlap_count(self, row, mode:str):
        model = self.spacy_ner_model
        def clean_ners(ner_list):
            return set([(ner[0].lower(), ner[1]) for ner in ner_list])
        prompt = model(row['prompt_text'])
        summary = model(row['text'])

        if "spacy" in str(model):
            prompt_ner = set([(token.text, token.label_) for token in prompt.ents])
            summary_ner = set([(token.text, token.label_) for token in summary.ents])
        elif "stanza" in str(model):
            prompt_ner = set([(token.text, token.type) for token in prompt.ents])
            summary_ner = set([(token.text, token.type) for token in summary.ents])
        else:
            raise Exception("Model not supported")

        prompt_ner = clean_ners(prompt_ner)
        summary_ner = clean_ners(summary_ner)

        intersecting_ners = prompt_ner.intersection(summary_ner)
        
        ner_dict = dict(Counter([ner[1] for ner in intersecting_ners]))
        
        if mode == "train":
            return ner_dict
        elif mode == "test":
            return {key: ner_dict.get(key) for key in self.ner_keys}

    
    def quotes_count(self, row):
        summary = row['text']
        text = row['prompt_text']
        quotes_from_summary = re.findall(r'"([^"]*)"', summary)
        if len(quotes_from_summary)>0:
            return [quote in text for quote in quotes_from_summary].count(True)
        else:
            return 0

    def spelling(self, text):
        
        wordlist=text.split()
        amount_miss = len(list(self.spellchecker.unknown(wordlist)))

        return amount_miss
    
    def add_spelling_dictionary(self, tokens: List[str]) -> List[str]:
        """dictionary update for pyspell checker and autocorrect"""
        self.spellchecker.word_frequency.load_words(tokens)
        self.speller.nlp_data.update({token:1000 for token in tokens})
    
    def run(self, 
            prompts: pd.DataFrame,
            summaries:pd.DataFrame,
            mode:str
        ) -> pd.DataFrame:
        
        # before merge preprocess
        prompts["prompt_length"] = prompts["prompt_text"].apply(
            lambda x: len(word_tokenize(x))
        )
        prompts["prompt_tokens"] = prompts["prompt_text"].apply(
            lambda x: word_tokenize(x)
        )

        summaries["summary_length"] = summaries["text"].apply(
            lambda x: len(word_tokenize(x))
        )
        summaries["summary_tokens"] = summaries["text"].apply(
            lambda x: word_tokenize(x)
        )
        
        # Add prompt tokens into spelling checker dictionary
        prompts["prompt_tokens"].apply(
            lambda x: self.add_spelling_dictionary(x)
        )
        
#         from IPython.core.debugger import Pdb; Pdb().set_trace()
        # fix misspelling
        summaries["fixed_summary_text"] = summaries["text"].progress_apply(
            lambda x: self.speller(x)
        )
        
        # count misspelling
        summaries["splling_err_num"] = summaries["text"].progress_apply(self.spelling)
        
        # merge prompts and summaries
        input_df = summaries.merge(prompts, how="left", on="prompt_id")

        # after merge preprocess
        input_df['length_ratio'] = input_df['summary_length'] / input_df['prompt_length']

        input_df['word_overlap_count'] = input_df.progress_apply(self.word_overlap_count, axis=1)
        input_df['bigram_overlap_count'] = input_df.progress_apply(
            self.ngram_co_occurrence,args=(2,), axis=1 
        )
        input_df['bigram_overlap_ratio'] = input_df['bigram_overlap_count'] / (input_df['summary_length'] - 1)
        
        input_df['trigram_overlap_count'] = input_df.progress_apply(
            self.ngram_co_occurrence, args=(3,), axis=1
        )
        input_df['trigram_overlap_ratio'] = input_df['trigram_overlap_count'] / (input_df['summary_length'] - 2)
        
        input_df['quotes_count'] = input_df.progress_apply(self.quotes_count, axis=1)
        
        return input_df.drop(columns=["summary_tokens", "prompt_tokens"])
    
preprocessor = Preprocessor(model_name=CFG.model_name)

In [ ]:
summaries_train.head()

In [ ]:
import pickle
with open(MODEL_DIR+'/pickled.pkl', 'rb') as f:
    train = pickle.load(f)
    test = pickle.load(f)

# train = preprocessor.run(prompts_train, summaries_train, mode="train")
test = preprocessor.run(prompts_test, summaries_test, mode="test")

# train.head()

In [ ]:
# train

In [ ]:
train = train.drop(columns = ['length_ratio'])
test = test.drop(columns = ['length_ratio'])

In [ ]:
gkf = GroupKFold(n_splits=CFG.n_splits)

for i, (_, val_index) in enumerate(gkf.split(train, groups=train["prompt_id"])):
    train.loc[val_index, "fold"] = i

train.head()

## Model Function Definition

In [ ]:
import shutil

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    rmse = mean_squared_error(labels, predictions, squared=False)
    return {"rmse": rmse}

def compute_mcrmse(eval_pred):
    """
    Calculates mean columnwise root mean squared error
    https://www.kaggle.com/competitions/commonlit-evaluate-student-summaries/overview/evaluation
    """
    preds, labels = eval_pred

    col_rmse = np.sqrt(np.mean((preds - labels) ** 2, axis=0))
    mcrmse = np.mean(col_rmse)

    return {
        "content_rmse": col_rmse[0],
        "wording_rmse": col_rmse[1],
        "mcrmse": mcrmse,
    }

def compt_score(content_true, content_pred, wording_true, wording_pred):
    content_score = mean_squared_error(content_true, content_pred)**(1/2)
    wording_score = mean_squared_error(wording_true, wording_pred)**(1/2)
    
    return (content_score + wording_score)/2

## Deberta Regressor

In [ ]:
class ContentScoreRegressor:
    def __init__(self, 
                model_name: str,
                model_dir: str,
                target: str,
                hidden_dropout_prob: float,
                attention_probs_dropout_prob: float,
                max_length: int,
                ):
        self.inputs = ["prompt_text", "prompt_title", "prompt_question", "fixed_summary_text"]
        self.input_col = "input"
        
        self.text_cols = [self.input_col] 
        self.target = target
        self.target_cols = [target]

        self.model_name = model_name
        self.model_dir = model_dir
        self.max_length = max_length
        
        self.tokenizer = AutoTokenizer.from_pretrained(INIT_MODEL)
        self.model_config = AutoConfig.from_pretrained(INIT_MODEL)
        
        self.model_config.update({
            "hidden_dropout_prob": hidden_dropout_prob,
            "attention_probs_dropout_prob": attention_probs_dropout_prob,
            "num_labels": 1,
            "problem_type": "regression",
        })
        
        seed_everything(seed=42)

        self.data_collator = DataCollatorWithPadding(
            tokenizer=self.tokenizer
        )


    def tokenize_function(self, examples: pd.DataFrame):
        labels = [examples[self.target]]
        tokenized_dict  = self.tokenizer(examples[self.input_col],
                            padding=False,
                            truncation=True,
                            max_length=self.max_length)
        input_ids = tokenized_dict.input_ids
        attention_mask = tokenized_dict.attention_mask 

        head_mask = []
        use_full = False
        for token in tokenized_dict.input_ids:            
            if token == self.tokenizer.sep_token_id:
                use_full = not use_full  

            head_mask.append(1 if use_full else .0) 



        # 'input_ids', 'token_type_ids', 'attention_mask'

        # from IPython.core.debugger import Pdb; Pdb().set_trace()


        return {
            'input_ids' : input_ids, 
            'attension_mask': attention_mask,
            'head_mask': head_mask, 
            "labels": labels,
        }
    
    def tokenize_function_test(self, examples: pd.DataFrame):
        tokenized_dict = self.tokenizer(examples[self.input_col],
                         padding=False,
                         truncation=True,
                         max_length=self.max_length)
        input_ids = tokenized_dict.input_ids
        attention_mask = tokenized_dict.attention_mask 

        head_mask = []
        use_full = False
        for token in tokenized_dict.input_ids:            
            if token == self.tokenizer.sep_token_id:
                use_full = not use_full  

            head_mask.append(1 if use_full else .0) 



        # 'input_ids', 'token_type_ids', 'attention_mask'

        # from IPython.core.debugger import Pdb; Pdb().set_trace()


        return {
            'input_ids' : input_ids, 
            'attension_mask': attention_mask,
            'head_mask': head_mask, 
        }
        
    def train(self, 
            fold: int,
            train_df: pd.DataFrame,
            valid_df: pd.DataFrame,
            batch_size: int,
            learning_rate: float,
            weight_decay: float,
            num_train_epochs: float,
            save_steps: int,
        ) -> None:
        """fine-tuning"""
        
        sep = self.tokenizer.sep_token
        train_df[self.input_col] = (
                    train_df["prompt_question"] + sep
                    + train_df["text"] + sep
                    + train_df["prompt_text"]
                  )

        valid_df[self.input_col] = (
                    valid_df["prompt_question"] + sep
                    + valid_df["text"] + sep
                    + valid_df["prompt_text"]
                  )

        
        train_df = train_df[[self.input_col] + self.target_cols]
        valid_df = valid_df[[self.input_col] + self.target_cols]
        
        model_content = AutoModelForSequenceClassification.from_pretrained(
            f"/kaggle/input/{self.model_name}", 
            config=self.model_config,
            ignore_mismatched_sizes=True
        )

        train_dataset = Dataset.from_pandas(train_df, preserve_index=False) 
        val_dataset = Dataset.from_pandas(valid_df, preserve_index=False) 
    
        train_tokenized_datasets = train_dataset.map(self.tokenize_function, batched=False)
        val_tokenized_datasets = val_dataset.map(self.tokenize_function, batched=False)

        # eg. "bert/fold_0/"
        model_fold_dir = os.path.join(self.model_dir, str(fold)) 
        
        training_args = TrainingArguments(
            output_dir=model_fold_dir,
            load_best_model_at_end=True, # select best model
            learning_rate=learning_rate,
            per_device_train_batch_size=batch_size,
            per_device_eval_batch_size=8,
            num_train_epochs=num_train_epochs,
            weight_decay=weight_decay,
            report_to='none',
            greater_is_better=False,
            save_strategy="steps",
            evaluation_strategy="steps",
            eval_steps=save_steps,
            save_steps=save_steps,
            metric_for_best_model="rmse",
            save_total_limit=1
        )

        trainer = Trainer(
            model=model_content,
            args=training_args,
            train_dataset=train_tokenized_datasets,
            eval_dataset=val_tokenized_datasets,
            tokenizer=self.tokenizer,
            compute_metrics=compute_metrics,
            data_collator=self.data_collator
        )

        trainer.train()
        
        shutil.rmtree(self.model_dir)
        
        model_content.save_pretrained(self.model_dir)
        self.tokenizer.save_pretrained(self.model_dir)

        
    def predict(self, 
                test_df: pd.DataFrame,
                fold: int,
               ):
        """predict content score"""
        
        sep = self.tokenizer.sep_token
        in_text = (
                    test_df["prompt_question"] + sep
                    + test_df["text"] + sep
                    + test_df["prompt_text"]
                  )
        test_df[self.input_col] = in_text

        test_ = test_df[[self.input_col]]
    
        test_dataset = Dataset.from_pandas(test_, preserve_index=False) 
        test_tokenized_dataset = test_dataset.map(self.tokenize_function_test, batched=False)

        model_content = AutoModelForSequenceClassification.from_pretrained(f"{self.model_dir}")
        model_content.eval()
        
        # e.g. "bert/fold_0/"
        model_fold_dir = os.path.join(self.model_dir, str(fold)) 
        model_fold_dir = "valid_log" #f"bert-{fold}"
#         print("model_fold_dir", model_fold_dir)
        
        test_args = TrainingArguments(
            output_dir=model_fold_dir,
            do_train = False,
            do_predict = True,
            per_device_eval_batch_size = 4,   
            dataloader_drop_last = False,
        )

        # init trainer
        infer_content = Trainer(
                      model = model_content, 
                      tokenizer=self.tokenizer,
                      data_collator=self.data_collator,
                      args = test_args)

        preds = infer_content.predict(test_tokenized_dataset)[0]

        return preds

In [ ]:
def train_by_fold(
        train_df: pd.DataFrame,
        model_name: str,
        target:str,
        save_each_model: bool,
        n_splits: int,
        batch_size: int,
        learning_rate: int,
        hidden_dropout_prob: float,
        attention_probs_dropout_prob: float,
        weight_decay: float,
        num_train_epochs: int,
        save_steps: int,
        max_length:int
    ):

    # delete old model files
    if os.path.exists(model_name):
        shutil.rmtree(model_name)
    
    os.mkdir(model_name)
        
    for fold in range(CFG.n_splits):
        print(f"fold {fold}:")
        
        train_data = train_df[train_df["fold"] != fold]
        valid_data = train_df[train_df["fold"] == fold]
        
        if save_each_model == True:
            model_dir =  f"{target}/{model_name}/fold_{fold}"
        else: 
            model_dir =  f"{model_name}/fold_{fold}"

        csr = ContentScoreRegressor(
            model_name=model_name,
            target=target,
            model_dir = model_dir, 
            hidden_dropout_prob=hidden_dropout_prob,
            attention_probs_dropout_prob=attention_probs_dropout_prob,
            max_length=max_length,
           )
        
        csr.train(
            fold=fold,
            train_df=train_data,
            valid_df=valid_data, 
            batch_size=batch_size,
            learning_rate=learning_rate,
            weight_decay=weight_decay,
            num_train_epochs=num_train_epochs,
            save_steps=save_steps,
        )

def validate(
    train_df: pd.DataFrame,
    target:str,
    save_each_model: bool,
    model_name: str,
    hidden_dropout_prob: float,
    attention_probs_dropout_prob: float,
    max_length : int
    ) -> pd.DataFrame:
    """predict oof data"""
    for fold in range(CFG.n_splits):
        print(f"fold {fold}:")
        
        valid_data = train_df[train_df["fold"] == fold]
        
        if save_each_model == True:
            model_dir =  f"{MODEL_DIR}/{target}/{model_name}/fold_{fold}"
        else: 
            model_dir =  f"{MODEL_DIR}/{model_name}/fold_{fold}"
        
#         print(model_dir, model_name)
        csr = ContentScoreRegressor(
            model_name=model_name,
            target=target,
            model_dir = model_dir,
            hidden_dropout_prob=hidden_dropout_prob,
            attention_probs_dropout_prob=attention_probs_dropout_prob,
            max_length=max_length,
           )
        
        pred = csr.predict(
            test_df=valid_data, 
            fold=fold
        )
        
        train_df.loc[valid_data.index, f"{target}_pred"] = pred

    return train_df
    
def predict(
    test_df: pd.DataFrame,
    target:str,
    save_each_model: bool,
    model_name: str,
    hidden_dropout_prob: float,
    attention_probs_dropout_prob: float,
    max_length : int
    ):
    """predict using mean folds"""

    for fold in range(CFG.n_splits):
        print(f"fold {fold}:")
        
        if save_each_model == True:
            model_dir =  f"{MODEL_DIR}/{target}/{model_name}/fold_{fold}"
        else: 
            model_dir =  f"{MODEL_DIR}/{model_name}/fold_{fold}"

        csr = ContentScoreRegressor(
            model_name=model_name,
            target=target,
            model_dir = model_dir, 
            hidden_dropout_prob=hidden_dropout_prob,
            attention_probs_dropout_prob=attention_probs_dropout_prob,
            max_length=max_length,
           )
        
        pred = csr.predict(
            test_df=test_df, 
            fold=fold
        )
        
        test_df[f"{target}_pred_{fold}"] = pred
    
    test_df[f"{target}"] = test_df[[f"{target}_pred_{fold}" for fold in range(CFG.n_splits)]].mean(axis=1)

    return test_df

In [ ]:
for target in ["content", "wording"]:
#     train_by_fold(
#         train,
#         model_name=CFG.model_name,
#         save_each_model=True,
#         target=target,
#         learning_rate=CFG.learning_rate,
#         hidden_dropout_prob=CFG.hidden_dropout_prob,
#         attention_probs_dropout_prob=CFG.attention_probs_dropout_prob,
#         weight_decay=CFG.weight_decay,
#         num_train_epochs=CFG.num_train_epochs,
#         n_splits=CFG.n_splits,
#         batch_size=CFG.batch_size,
#         save_steps=CFG.save_steps,
#         max_length=CFG.max_length
#     )
    
    print("[validate]")
    train = validate(
        train,
        target=target,
        save_each_model=True,
        model_name=CFG.model_name,
        hidden_dropout_prob=CFG.hidden_dropout_prob,
        attention_probs_dropout_prob=CFG.attention_probs_dropout_prob,
        max_length=CFG.max_length
    )

    rmse = mean_squared_error(train[target], train[f"{target}_pred"], squared=False)
    print(f"cv {target} rmse: {rmse}")

    print("[test]")
    test = predict(
        test,
        target=target,
        save_each_model=True,
        model_name=CFG.model_name,
        hidden_dropout_prob=CFG.hidden_dropout_prob,
        attention_probs_dropout_prob=CFG.attention_probs_dropout_prob,
        max_length=CFG.max_length
    )

In [ ]:
# !rm -r wording content

In [ ]:
train.head()

# Add Features

In [ ]:
wd = pd.read_csv('/kaggle/input/worddifficulty/WordDifficulty.csv')
dic = dict(zip(wd['Word'], wd['I_Zscore']))

In [ ]:
def difficulty(data) :
    words = word_tokenize(data['text'])
#     s = ['``','\'\'','.',',']
#     stop_words = set(stopwords.words('english') + s)
#     filtered_words = [word for word in words if word.lower() not in stop_words]
    filtered_words = words
    score = 0
    num = 0
    sep = 0
    for e in filtered_words:
        if e in dic:
            score += dic[e]
            num+=1
        elif e == '.' or e == ',' :
            sep+=1
        else:
            pass
#             print(e,"**")

    nn = max(1, len(filtered_words))
    sep = max(1, sep)
    num = max(1, num)
    return score/num, score/nn, nn/sep, (nn-num)/nn



#         prompts["prompt_tokens"] = prompts["prompt_text"].apply(
#             lambda x: word_tokenize(x)
#         )
labels = ['difficulty0', 'difficulty1', 'ave_text_len', 'unknown_words']
train[labels]=train.apply(lambda x:difficulty(x),axis=1, result_type='expand')
test[labels]=test.apply(lambda x:difficulty(x),axis=1, result_type='expand')

In [ ]:
from readability import Readability

In [ ]:
no_score = -10000
def rscore(data) :
    txt = data['text']
    words = word_tokenize(txt)
    n = len(words) + 1
    if n == 0 : 
        return no_score, no_score, no_score, no_score, no_score, no_score, no_score, no_score
    tot = n
    new = txt
    while tot < 200 :
        new += " " + txt
        tot += n
    r = Readability(new)
    try :
        ret = (r.flesch_kincaid().score, r.flesch().score, r.gunning_fog().score,
               r.coleman_liau().score,r.dale_chall().score, r.ari().score,
               r.linsear_write().score, r.spache().score)
    except:
        return no_score, no_score, no_score, no_score, no_score, no_score, no_score, no_score
    return ret

labels = ['flesch_kincaid', 'flesch', 'gunning_fog', 'coleman_liau',
         'dale_chall','ari','linsear_write','spache']

train[labels]=train.progress_apply(lambda x:rscore(x),axis=1, result_type='expand')
test[labels]=test.progress_apply(lambda x:rscore(x),axis=1, result_type='expand')

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import nltk

dic = dict()
for i in range(len(prompts_train)) :
    pid = prompts_train['prompt_id'][i]
    txt = prompts_train['prompt_text'][i]
    original_tokens = nltk.word_tokenize(txt)
    original_text = ' '.join(original_tokens)
    dic[pid] = original_text
for i in range(len(prompts_test)) :
    pid = prompts_test['prompt_id'][i]
    txt = prompts_test['prompt_text'][i]
    original_tokens = nltk.word_tokenize(txt)
    original_text = ' '.join(original_tokens)
    dic[pid] = original_text
    
dic.keys()

def cosine_sim(data):
    original_text = dic[data['prompt_id']]
    summary = data['fixed_summary_text']
#     original_tokens = nltk.word_tokenize(original_text)
    summary_tokens = nltk.word_tokenize(summary)

    # トークンを結合して文に戻す
#     original_text = ' '.join(original_tokens)
    summary = ' '.join(summary_tokens)

    # CountVectorizerを使用して文をベクトル化
    vectorizer = CountVectorizer().fit_transform([original_text, summary])

    # コサイン類似度を計算
    cosine_scores = cosine_similarity(vectorizer)

    # 要約と元の文章の類似度を表示
    similarity_score = cosine_scores[0][1]
    
    return similarity_score

In [ ]:
# cosine_sim(train.iloc[0])
train['cos_sim']=train.progress_apply(lambda x:cosine_sim(x),axis=1, result_type='expand')
test['cos_sim']=test.progress_apply(lambda x:cosine_sim(x),axis=1, result_type='expand')

In [ ]:
import textstat
def txts(data):
    text = data['text']
    return (
        textstat.flesch_reading_ease(text),
        textstat.flesch_kincaid_grade(text),
        textstat.gunning_fog(text),
        textstat.smog_index(text),
        textstat.automated_readability_index(text),
        textstat.coleman_liau_index(text),
        textstat.linsear_write_formula(text),
        textstat.dale_chall_readability_score(text),
        textstat.text_standard(text, float_output=True),
        textstat.reading_time(text, ms_per_char=14.69),
        textstat.syllable_count(text),
        textstat.lexicon_count(text, removepunct=True),
        textstat.sentence_count(text),
        textstat.char_count(text, ignore_spaces=True),
        textstat.letter_count(text, ignore_spaces=True),
        textstat.monosyllabcount(text)
    )

sample = txts(train.iloc[1])
print(sample)
labels = [f'f{i}' for i in range(len(sample))]
train[labels]=train.progress_apply(lambda x:txts(x),axis=1, result_type='expand')
test[labels]=test.progress_apply(lambda x:txts(x),axis=1, result_type='expand')

## Predict

In [ ]:
targets = ['content','wording']
for target in targets:
    test[target] = test[[f"{target}_pred_{fold}" for fold in range(CFG.n_splits)]].mean(axis=1)

## Create Submission file

In [ ]:
sample_submission

In [ ]:
test[["student_id", "content", "wording"]]

In [ ]:
test[["student_id", "content", "wording"]].to_csv("submission.csv", index=False)

## Summary

CV result is like this.

| | content rmse |wording rmse | mcrmse | LB| |
| -- | -- | -- | -- | -- | -- |
|baseline| 0.494 | 0.630 | 0.562 | 0.509 | [link](https://www.kaggle.com/code/tsunotsuno/debertav3-baseline-content-and-wording-models)|
| use title and question field | 0.476| 0.619 | 0.548 | 0.508 | [link](https://www.kaggle.com/code/tsunotsuno/debertav3-w-prompt-title-question-fields) |
| Debertav3 + LGBM | 0.451 | 0.591 | 0.521 | 0.461 | [link](https://www.kaggle.com/code/tsunotsuno/debertav3-lgbm-with-feature-engineering) |
| Debertav3 + LGBM with spell autocorrect | 0.448 | 0.581 | 0.514 | 0.459 |nogawanogawa's original code
| Debertav3 + LGBM with spell autocorrect and tuning | 0.442 | 0.566 | 0.504 | 0.453 | this notebook |

The CV values improved slightly, and the LB value is improved.